In [110]:
import pandas as pd, matplotlib.pyplot as plt, numpy as np
import csv
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
%matplotlib inline

In [248]:
nba = pd.read_csv('AllStarNBA')
nba_2016=pd.read_csv('NBA_2016')

In [249]:
nba_2015= nba[nba.season_end == 2015]
nbaHistoric= nba[nba.season_end <= 2014]

In [250]:
nbaHistoric2= nba[nba.season_end <= 2015]

In [251]:
d = nbaHistoric
d.info()

d_2015 = nba_2015
d_2015.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10965 entries, 0 to 10964
Data columns (total 53 columns):
Unnamed: 0      10965 non-null int64
player          10965 non-null object
pos             10965 non-null object
age             10965 non-null int64
bref_team_id    10965 non-null object
g               10965 non-null int64
gs              10965 non-null int64
mp              10965 non-null float64
fg              10965 non-null float64
fga             10965 non-null float64
fg_             10965 non-null float64
x3p             10965 non-null float64
x3pa            10965 non-null float64
x3p_            10965 non-null float64
x2p             10965 non-null float64
x2pa            10965 non-null float64
x2p_            10965 non-null float64
efg             10965 non-null float64
ft              10965 non-null float64
fta             10965 non-null float64
ft_             10965 non-null float64
orb             10965 non-null float64
drb             10965 non-null float64
trb  

In [252]:
col_u = ["g","gs","mp","fg","fga","fg_","x3p","x3pa","x3p_","x2p","x2pa","x2p_","ft","fta","ft_","orb","drb","trb","ast","stl","blk","tov","pf","pts"]

In [253]:
features= d[col_u]
features_2015 = d_2015[col_u]
#d.team.replace('None',0,inplace=True)
#labels=pd.to_numeric(d.team,errors='coerce')

In [254]:
#d.team= np.where(d.team != 'None',1,0)
labels = d.team

In [255]:
labels.unique()

array(['None', '2.0', '3.0', '1.0'], dtype=object)

In [256]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.3, random_state=1)
train = pd.DataFrame(data=features_train, columns=[col_u])
train['team'] = labels_train
test = pd.DataFrame(data=features_test, columns=[col_u])
test['team'] = labels_test


# Random Forest

In [305]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import cross_val_score

clf = RandomForestClassifier(n_estimators=100,max_features='auto',min_samples_split=1)

clf = clf.fit(features_train,labels_train)

pred = clf.predict(features_test)

print "Accuracy =", accuracy_score(pred, labels_test)*100,"%"
scores = cross_val_score(clf, features_train, labels_train)
print scores.mean()
pd.DataFrame({'feature':col_u, 'importance':clf.feature_importances_}).sort_values(by='importance',ascending=False)

Accuracy = 98.2066869301 %
0.978500912553


,feature,importance
23,pts,0.076683
3,fg,0.074965
13,fta,0.068310
9,x2p,0.052674
12,ft,0.052533
16,drb,0.051549
18,ast,0.051002
2,mp,0.046286
4,fga,0.043374
10,x2pa,0.042138


In [258]:
#features_2015 = d_2015[col_u]
d_2015['pred2015']=clf.predict(features_2015)
d_2015['rfprd']=clf.predict_proba(features_2015)[:, 1]
#pred = clf.predict(features_test)
df=d_2015[['player','team','pred2015','rfprd']]

C:\Users\mahesh\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\mahesh\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [278]:
df[df.team!='None'].sort_values(by='team')

,player,team,pred2015,rfprd
11202,LeBron James,1.0,2.0,0.35
11077,Stephen Curry,1.0,None,0.28
11082,Anthony Davis,1.0,None,0.23
11201,LeBron James,1.0,2.0,0.35
11131,Marc Gasol,1.0,None,0.02
11163,James Harden,1.0,3.0,0.21
10973,LaMarcus Aldridge,2.0,None,0.10
11328,Chris Paul,2.0,None,0.25
11437,Russell Westbrook,2.0,1.0,0.31
11132,Pau Gasol,2.0,3.0,0.20


# Adaboost

In [279]:

from sklearn.ensemble import AdaBoostClassifier
adaclf = AdaBoostClassifier(n_estimators=10)

adaclf = adaclf.fit(features_train,labels_train)

adapred = adaclf.predict(features_test)

print "Accuracy =", accuracy_score(adapred, labels_test)*100,"%"
adascores = cross_val_score(adaclf, features_train, labels_train)
print adascores.mean()
adaclf.feature_importances_

Accuracy = 97.5075987842 %
0.968469322783


array([ 0. ,  0. ,  0.1,  0. ,  0. ,  0.1,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  0. ,  0. ,  0.1,  0. ,  0.2,  0. ,  0.1,  0. ,
        0.1,  0.3])

In [280]:
#features_2015 = d_2015[col_u]
d_2015['adapredict']=adaclf.predict(features_2015)
d_2015['adaprb']=adaclf.predict_proba(features_2015)[:, 1]
#pred = clf.predict(features_test)
df=d_2015[['player','team','pred2015','rfprd','adapredict','adaprb']]

C:\Users\mahesh\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\mahesh\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [283]:
df[df.adapredict!='None'].sort_values(by='team')

,player,team,pred2015,rfprd,adapredict,adaprb
11082,Anthony Davis,1.0,None,0.23,1.0,0.228756
11437,Russell Westbrook,2.0,1.0,0.31,1.0,0.271869


# KNN

In [284]:
from sklearn.neighbors import KNeighborsClassifier
knnclf = KNeighborsClassifier(n_neighbors=20)

knnclf = knnclf.fit(features_train,labels_train)

knnpred = knnclf.predict(features_test)

print "Accuracy =", accuracy_score(knnpred, labels_test)*100,"%"
knnscores = cross_val_score(knnclf, features_train, labels_train)
print knnscores.mean()


Accuracy = 97.5987841945 %
0.972247043811


In [285]:
#features_2015 = d_2015[col_u]
d_2015['knnpred']=knnclf.predict(features_2015)
d_2015['knnprb']=knnclf.predict_proba(features_2015)[:, 1]
#pred = clf.predict(features_test)
df=d_2015[['player','team','pred2015','rfprd','adapredict','adaprb','knnpred','knnprb']]

C:\Users\mahesh\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\mahesh\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [304]:
df[df.team!='None'].sort_values(by='team')

,player,team,pred2015,rfprd,adapredict,adaprb,knnpred,knnprb,dtpred,dtprb,svcpred
11202,LeBron James,1.0,2.0,0.35,None,0.245599,None,0.15,1.0,0,0
11077,Stephen Curry,1.0,None,0.28,None,0.233020,None,0.10,2.0,1,0
11082,Anthony Davis,1.0,None,0.23,1.0,0.228756,None,0.15,1.0,0,0
11201,LeBron James,1.0,2.0,0.35,None,0.245599,None,0.15,1.0,0,0
11131,Marc Gasol,1.0,None,0.02,None,0.242525,None,0.05,2.0,1,0
11163,James Harden,1.0,3.0,0.21,None,0.244127,None,0.00,None,0,0
10973,LaMarcus Aldridge,2.0,None,0.10,None,0.245599,None,0.15,3.0,0,0
11328,Chris Paul,2.0,None,0.25,None,0.253645,None,0.25,1.0,0,0
11437,Russell Westbrook,2.0,1.0,0.31,1.0,0.271869,2.0,0.35,1.0,0,0
11132,Pau Gasol,2.0,3.0,0.20,None,0.237343,None,0.15,None,0,0


# Decision tree

In [307]:
from sklearn.tree import DecisionTreeClassifier

dtclf = DecisionTreeClassifier(min_samples_split=1)

dtclf = dtclf.fit(features_train, labels_train)

dtpred = dtclf.predict(features_test)

print "accuracy =", dtclf.score(features_test, labels_test) * 100, '%'
dtscores = cross_val_score(dtclf, features_train, labels_train)
print dtscores.mean()
pd.DataFrame({'feature':col_u, 'importance':dtclf.feature_importances_}).sort_values(by='importance',ascending=False)

accuracy = 96.8996960486 %
0.964169444163


,feature,importance
23,pts,0.266021
18,ast,0.077099
14,ft_,0.069101
20,blk,0.052851
16,drb,0.051472
9,x2p,0.045174
5,fg_,0.041445
7,x3pa,0.039929
21,tov,0.038244
8,x3p_,0.036040


In [289]:
#features_2015 = d_2015[col_u]
d_2015['dtpred']=dtclf.predict(features_2015)
d_2015['dtprb']=dtclf.predict_proba(features_2015)[:, 1]
#pred = clf.predict(features_test)
df=d_2015[['player','team','pred2015','rfprd','adapredict','adaprb','knnpred','knnprb','dtpred','dtprb']]

C:\Users\mahesh\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\mahesh\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [301]:
df[df.dtpred!='None'].sort_values(by='team')

,player,team,pred2015,rfprd,adapredict,adaprb,knnpred,knnprb,dtpred,dtprb,svcpred
11202,LeBron James,1.0,2.0,0.35,None,0.245599,None,0.15,1.0,0,0
11201,LeBron James,1.0,2.0,0.35,None,0.245599,None,0.15,1.0,0,0
11131,Marc Gasol,1.0,None,0.02,None,0.242525,None,0.05,2.0,1,0
11077,Stephen Curry,1.0,None,0.28,None,0.233020,None,0.10,2.0,1,0
11082,Anthony Davis,1.0,None,0.23,1.0,0.228756,None,0.15,1.0,0,0
10973,LaMarcus Aldridge,2.0,None,0.10,None,0.245599,None,0.15,3.0,0,0
11328,Chris Paul,2.0,None,0.25,None,0.253645,None,0.25,1.0,0,0
11437,Russell Westbrook,2.0,1.0,0.31,1.0,0.271869,2.0,0.35,1.0,0,0
11069,DeMarcus Cousins,2.0,None,0.13,None,0.253606,None,0.15,3.0,0,0
10974,LaMarcus Aldridge,2.0,None,0.10,None,0.245599,None,0.15,3.0,0,0


# Naive Bayes

In [181]:
from sklearn.naive_bayes import GaussianNB
nbclf = GaussianNB()

nbclf=nbclf.fit(features_train,labels_train)

nbpred=nbclf.predict(features_test)

print "accuracy =", nbclf.score(features_test, labels_test) * 100, '%'
nbscores = cross_val_score(nbclf, features_train, labels_train)
print nbscores.mean()


accuracy = 85.1671732523 %
0.852770203946


# SVC

In [183]:
from sklearn.svm import SVC
svcclf = SVC(kernel='rbf')

svcclf = svcclf.fit(features_train, labels_train)

svcpred = svcclf.predict(features_test)

print "accuracy =", svcclf.score(features_test, labels_test) * 100, '%'
svcscores = cross_val_score(svcclf, features_train, labels_train)
print svcscores.mean()


accuracy = 97.8419452888 %
0.973549839738
